# Hyper Parameter Tuning
- hyper parameter : 모델 설정과 관련해 직접 지정할 수 있는 매개변수 
- model parameter : 회귀계수(가중치), 절편 등 모델의 학습 대상이 되는 변수 

### GridSearchCV

In [12]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

iris_input, iris_target = load_iris(return_X_y=True)
knn = KNeighborsClassifier()

params = {
    'n_naighbors' : range(1, 13, 2)
}

# 첫 번째 인자: 모델
# 두 번째 인자: 테스트 할 파라미터 (딕셔너리)
# scoring: 평가 지표 (accuracy, precision, recall, f1)
# cv: 반복 횟수
grid = GridSearchCV(knn, params, scoring='accuracy', cv=5)
grid.fit(iris_input, iris_target)

print('최적의 파라미터:', grid.best_params_)
print('최적화된 모델 객체:', grid.best_estimator_)
print('최적화된 점수: ', grid.best_score_)

ValueError: Invalid parameter 'n_naighbors' for estimator KNeighborsClassifier(). Valid parameters are: ['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'].

In [ ]:
best_knn = grid.best_estimator_
best_knn.score(iris_input, iris_target)

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

: 

### RandomSearchCV 
- 하이퍼 파라미터의 값 목록이나 값의 범위를 제공하는데, 이 범위 중에 랜덤하게 값을 뽑아내 최적의 하이퍼 파라미터 조합을 찾는다. 
    - 탐색 범위가 넓을 때 짧은 시간 내에 좋은 결과를 얻을 수 있다. 
    - 랜덤하게 값을 추출해 계산하므로, 전역 최적값을 놓칠 수 있다. 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

knn = KNeighborsClassifier()

params = {
    'n_neighbors' : range(1, 1000, 2)
}

rd_search = RandomizedSearchCV(knn, params, cv=5, n_iter=10, random_state=0)
rd_search.fit(iris_input, iris_target)

print('최적의 파라미터: ', rd_search.best_params_)
print('최적화된 모델 객체: ', rd_search.best_estimator_)
print('최적화된 점수: ', rd_search.best_score_ )

최적의 파라미터:  {'n_neighbors': 31}
최적화된 모델 객체:  KNeighborsClassifier(n_neighbors=31)
최적화된 점수:  0.9333333333333332


c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\sklearn\model_selection\_validation.py:953: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\sklearn\model_selection\_validation.py", line 942, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\sklearn\metrics\_scorer.py", line 492, in __call__
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\sklearn\neighbors\_classification.py", line 446, in score
    return super().score(X, y, sample_weight)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\skle

### HyperOpt

깃 설명 확인

In [ ]:
!pip install hyperopt

  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.6 MB 728.2 kB/s eta 0:00:02
   ------------------- -------------------- 0.8/1.6 MB 798.0 kB/s eta 0:00:01
   -------------------------- ------------- 1.0/1.6 MB 867.1 kB/s eta 0:00:01
   --------------------------------- ------ 1.3/1.6 MB 894.7 kB/s eta 0:00:01
   --------------------------------- ------ 1.3/1.6 MB 894.7 kB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 918.8 kB/s eta 0:00:00
Using cached networkx-3.5-py3-none-any.whl (2.0 MB)

   ------ --------------------------------- 1/6 [tqdm]
   ------------- -------------------------- 2/6 [networkx]
   ------------- ---------

In [13]:
from hyperopt import hp 

# 검색 공간 : 하이퍼파라미터 최적화 시 사용하는 범위 지정(검색 공간)
search_space = {
    'x' : hp.quniform('x', -10, 10, 1),
    'y' : hp.quniform('y', -15, 15, 1)
}


In [14]:
import hyperopt 

# 목적 함수 
def objective(search_space) :
    x = search_space['x']
    y = search_space['y']
    return { 
        'loss' : x ** 2 + 20 * y, 
        'status' : hyperopt.STATUS_OK    # STATUS_OK 현재의 계산이 잘 끝났는지 물어보는 것 
    }


In [15]:
from hyperopt import fmin, tpe, Trials   # 목적함수에 대한 loss 를 최고화 하는 함수

trials = Trials()

best_val = fmin(            # 함수 호출 구문
    fn = objective,         # 목적함수
    space = search_space,   
    algo = tpe.suggest,     # 베이지안 함수 기반 탐색 방법
    max_evals = 500,        # 반복횟수
    trials = trials         # 목적함수의 값을 최소화 시키는 값을 찾아가면서 해당 결과 저장
)
best_val

100%|██████████| 500/500 [00:05<00:00, 85.77trial/s, best loss: -300.0] 


{'x': np.float64(-0.0), 'y': np.float64(-15.0)}

In [16]:
trials.results   # loss, status 로 구성된 딕셔너리 - 목적함수에서 봄?

[{'loss': 189.0, 'status': 'ok'},
 {'loss': 289.0, 'status': 'ok'},
 {'loss': 109.0, 'status': 'ok'},
 {'loss': -199.0, 'status': 'ok'},
 {'loss': 201.0, 'status': 'ok'},
 {'loss': -115.0, 'status': 'ok'},
 {'loss': -119.0, 'status': 'ok'},
 {'loss': -84.0, 'status': 'ok'},
 {'loss': 121.0, 'status': 'ok'},
 {'loss': 289.0, 'status': 'ok'},
 {'loss': 56.0, 'status': 'ok'},
 {'loss': 264.0, 'status': 'ok'},
 {'loss': 64.0, 'status': 'ok'},
 {'loss': -179.0, 'status': 'ok'},
 {'loss': -171.0, 'status': 'ok'},
 {'loss': 76.0, 'status': 'ok'},
 {'loss': 81.0, 'status': 'ok'},
 {'loss': -20.0, 'status': 'ok'},
 {'loss': -56.0, 'status': 'ok'},
 {'loss': 196.0, 'status': 'ok'},
 {'loss': -199.0, 'status': 'ok'},
 {'loss': -244.0, 'status': 'ok'},
 {'loss': -124.0, 'status': 'ok'},
 {'loss': -264.0, 'status': 'ok'},
 {'loss': -184.0, 'status': 'ok'},
 {'loss': -79.0, 'status': 'ok'},
 {'loss': -164.0, 'status': 'ok'},
 {'loss': -251.0, 'status': 'ok'},
 {'loss': -231.0, 'status': 'ok'},
 {'lo

In [17]:
trials.vals   # 하이퍼파라미터 값. 적용했던 값들을 리스트로 가지고 있음. 

{'x': [np.float64(3.0),
  np.float64(7.0),
  np.float64(-3.0),
  np.float64(9.0),
  np.float64(-9.0),
  np.float64(-5.0),
  np.float64(-9.0),
  np.float64(-6.0),
  np.float64(-1.0),
  np.float64(-7.0),
  np.float64(4.0),
  np.float64(2.0),
  np.float64(-8.0),
  np.float64(9.0),
  np.float64(-7.0),
  np.float64(-6.0),
  np.float64(9.0),
  np.float64(0.0),
  np.float64(-2.0),
  np.float64(-4.0),
  np.float64(9.0),
  np.float64(6.0),
  np.float64(6.0),
  np.float64(6.0),
  np.float64(6.0),
  np.float64(1.0),
  np.float64(4.0),
  np.float64(7.0),
  np.float64(3.0),
  np.float64(7.0),
  np.float64(7.0),
  np.float64(4.0),
  np.float64(4.0),
  np.float64(2.0),
  np.float64(5.0),
  np.float64(-0.0),
  np.float64(5.0),
  np.float64(10.0),
  np.float64(2.0),
  np.float64(-2.0),
  np.float64(8.0),
  np.float64(3.0),
  np.float64(1.0),
  np.float64(5.0),
  np.float64(-3.0),
  np.float64(8.0),
  np.float64(-1.0),
  np.float64(10.0),
  np.float64(3.0),
  np.float64(5.0),
  np.float64(1.0),
  np.flo

- hyperopt를 활용한 XGBoost 하이퍼 파라미터 튜닝

In [18]:
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from hyperopt import fmin, tpe, Trials, hp
import hyperopt

# 0. 데이터 로드 및 분리 (유방암 데이터 로드)
data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, random_state=0)


# 1. 검색 공간
# n_estimators, max_depth, learning_rate, colsample_bytree
search_space = {
    "n_estimators": hp.quniform('n_estimators', 100, 500, 100),
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'learning_rate' : hp.uniform('learning_rate', 0.01, 0.2), 
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1)
}

# 2. 목적 함수 
# cross_val_score - accuracy(평가지표) : 이진분류임. / accuracy의 경우, 값이 클 수록 성능이 좋다는 뜻 -> (-) 음수로 변환해줘야 성능지표로 쓸 수 있음. 
def xgb_objective(ss) :
    xgb_clf = XGBClassifier(
        n_estimators = int(ss['n_estimators']), 
        max_depth = int(ss['max_depth']), 
        learning_rate = ss['learning_rate'], 
        colsample_bytree = ss['colsample_bytree']
    )
    mean_acc = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3).mean()
    return { 
        'loss' : -1 * mean_acc,
        'status' : hyperopt.STATUS_OK   
    }


# 3. Trials() + fmin()
# 50 반복
trials = Trials()

best = fmin(     
    fn = xgb_objective,
    space = search_space,   
    algo = tpe.suggest,     # 베이지안 함수 기반 탐색 방법
    max_evals = 50,        # 반복횟수
    trials = trials         # 목적함수의 값을 최소화 시키는 값을 찾아가면서 해당 결과 저장
) 
best

100%|██████████| 50/50 [00:13<00:00,  3.76trial/s, best loss: -0.9671361502347416]


{'colsample_bytree': np.float64(0.6403349404192222),
 'learning_rate': np.float64(0.1351017557199938),
 'max_depth': np.float64(4.0),
 'n_estimators': np.float64(400.0)}

----------

### Optuna

In [ ]:
!pip install optuna

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 1.0 MB/s eta 0:00:02
   --------- ------------------------------ 0.5/2.1 MB 1.0 MB/s eta 0:00:02
   -------------- ------------------------- 0.8/2.1 MB 817.9 kB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 931.8 kB/s eta 0:00:02
   ------------------------ --------------- 1.3/2.1 MB 1.0 MB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 1.0 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 1.1 MB/s eta 0:00:00

   ------ --------------------------------- 1/6 [greenlet]
   -------------------- ------------------- 3/6 [sql

In [19]:
import optuna

# 목적 함수 
def objective(trial) : 
    x = trial.suggest_uniform('x', -10, 10)   # 목적함수 안쪽에 범위 정의 
    y = trial.suggest_uniform('y', -15, 15)
    return (x - 3) ** 2 + (y + 5) ** 2

# 스터디 생성
study = optuna.create_study(direction='minimize')  # direction을 최소화 시키는 방향으로 최적화를 진행함

# 최적화 실행
study.optimize(objective, n_trials=500)

[I 2025-08-11 10:21:07,951] A new study created in memory with name: no-name-4a02262a-239a-48d5-b611-65c34247a8ef
C:\Users\Playdata\AppData\Local\Temp\ipykernel_27956\2501295960.py:5: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

C:\Users\Playdata\AppData\Local\Temp\ipykernel_27956\2501295960.py:6: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2025-08-11 10:21:07,953] Trial 0 finished with value: 208.09434365760814 and parameters: {'x': -7.004717928901338, 'y': 5.392303056624563}. Best is trial 0 with value: 208.09434365760814.
[I 2025-08-11 10:21:07,954] Trial 1 finished with value: 10.675579272058835 and parameters: {'x': 6.033542752230346, 'y': -3.786246465525405}. Best is trial 1 with value: 10.675

In [20]:
study.best_value

0.003361844368758654

In [21]:
study.best_params

{'x': 3.0417248993211032, 'y': -5.040260118546801}

In [22]:
import optuna.visualization as vis

vis.plot_param_importances(study).show()

In [23]:
vis.plot_optimization_history(study).show()   # 최적화를 진행하는 과정을 시각화해줌. 

- optuna를 활용한 XGBoost 하이퍼 파라미터 튜닝

In [26]:
# 1. 목적 함수 
def xgb_optuna_objective(trial) : 
    params = {
        "n_estimators": trial.suggest_int('n_estimators', 100, 500, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 10, 1),
        'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.2), 
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1)
    }
    xgb_clf = XGBClassifier(**params)
    return cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3).mean()

# 2. study 객체 생성 -> 최적화
study = optuna.create_study(direction='maximize')  # direction을 최소화 시키는 방향으로 최적화를 진행함
study.optimize(xgb_optuna_objective, n_trials=50)

# 3. 결과 출력
print(study.best_value)
print(study.best_params)

[I 2025-08-11 10:24:44,209] A new study created in memory with name: no-name-9c9d2fe3-8261-4d6d-aa3d-3a27ec22f35b
C:\Users\Playdata\AppData\Local\Temp\ipykernel_27956\3633044676.py:4: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https://github.com/optuna/optuna/releases/tag/v3.5.0 for details.

C:\Users\Playdata\AppData\Local\Temp\ipykernel_27956\3633044676.py:5: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments i

0.9671361502347416
{'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.06964275018292802, 'colsample_bytree': 0.6549227190729764}


## HyperOpt vs Optuna

- HyperOpt
    - 'colsample_bytree': np.float64(0.6403349404192222),
    - 'learning_rate': np.float64(0.1351017557199938),
    - 'max_depth': np.float64(4.0),
    - 'n_estimators': np.float64(400.0)

- Optuna
    - 'colsample_bytree': 0.6549227190729764, 
    - 'learning_rate': 0.06964275018292802,
    - 'max_depth': 5,
    - 'n_estimators': 200,   

In [29]:
from sklearn.metrics import classification_report

xgb_hopt = XGBClassifier(
    n_estimators = 500, 
    max_depth = 6, 
    leaning_rate = 0.17, 
    colsample_bytree = 0.5
)

xgb_optuna = XGBClassifier(
    n_estimators = 500, 
    max_depth = 7, 
    leaning_rate = 0.14, 
    colsample_bytree = 0.53
)

xgb_hopt.fit(X_train, y_train)
xgb_optuna.fit(X_train, y_train)

hopt_pred = xgb_hopt.predict(X_test)
optuna_pred = xgb_optuna.predict(X_test)

print('HyperOpt 최적 파라미터 적용')
print(classification_report(y_test, hopt_pred))
print('Optuna 최적 파라미터 적용')
print(classification_report(y_test, optuna_pred))

HyperOpt 최적 파라미터 적용
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        53
           1       0.99      0.99      0.99        90

    accuracy                           0.99       143
   macro avg       0.99      0.99      0.99       143
weighted avg       0.99      0.99      0.99       143

Optuna 최적 파라미터 적용
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        53
           1       0.99      0.99      0.99        90

    accuracy                           0.99       143
   macro avg       0.99      0.99      0.99       143
weighted avg       0.99      0.99      0.99       143



c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\xgboost\training.py:183: UserWarning:

[10:33:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "leaning_rate" } are not used.




### 하이퍼파라미터 찾는 방법
1. 일일 대조 
2. cv 교차 검증 방식으로 최적의 파라미터 찾기 
3. hyperopt
4. optuna - randomsearchcv와 유사하다고 볼수 있음. 